## Deliverable 2. Create a Customer Travel Destinations Map.

## Dependencies and keys imported correctly

In [27]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind,Description,Date
0,0,Tocopilla,CL,-22.0920,-70.1979,66.11,67,76,3.58,broken clouds,2021-07-11 18:46:46
1,1,Vestmannaeyjar,IS,63.4427,-20.2734,56.39,82,40,16.11,scattered clouds,2021-07-11 18:46:46
2,2,Rikitea,PF,-23.1203,-134.9692,73.69,83,38,21.85,light rain,2021-07-11 18:46:47
3,3,Zumpahuacan,MX,18.9000,-99.5500,77.11,66,94,5.30,overcast clouds,2021-07-11 18:46:47
4,4,Barrow,US,71.2906,-156.7887,33.82,79,90,11.50,haze,2021-07-11 18:46:47


## Write input statements to prompt user to enter min/max temp.

In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

## Use the loc method to filter the DataFrame for temp critera

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind,Description,Date
3,3,Zumpahuacan,MX,18.9000,-99.5500,77.11,66,94,5.30,overcast clouds,2021-07-11 18:46:47
5,5,Thompson,CA,55.7435,-97.8558,84.36,48,20,8.05,smoke,2021-07-11 18:46:48
7,7,Arraial Do Cabo,BR,-22.9661,-42.0278,78.76,64,0,11.50,clear sky,2021-07-11 18:46:48
12,12,Atuona,PF,-9.8000,-139.0333,77.63,80,57,25.48,light rain,2021-07-11 18:46:50
14,14,Oyama,JP,36.3000,139.8000,76.59,93,10,2.53,clear sky,2021-07-11 18:46:50
15,15,Kahului,US,20.8947,-156.4700,80.96,70,20,12.66,few clouds,2021-07-11 18:46:51
18,18,Bengkulu,ID,-3.8004,102.2655,76.08,88,100,4.72,light rain,2021-07-11 18:46:52
22,22,Sechura,PE,-5.5569,-80.8222,76.12,61,97,12.06,overcast clouds,2021-07-11 18:46:53
23,23,Albany,US,42.6001,-73.9662,75.36,88,100,2.59,light rain,2021-07-11 18:45:20
24,24,Corn Island,NI,12.1694,-83.0418,80.40,78,100,13.00,overcast clouds,2021-07-11 18:46:53


## Determine if there are empty rows and drop as necessary

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        198
City           198
Country        198
Lat            198
Lng            198
Max Temp       198
Humidity       198
Cloudiness     198
Wind           198
Description    198
Date           198
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# None

## Create a new DataFrame called hotel_df

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Zumpahuacan,MX,77.11,overcast clouds,18.9000,-99.5500,
5,Thompson,CA,84.36,smoke,55.7435,-97.8558,
7,Arraial Do Cabo,BR,78.76,clear sky,-22.9661,-42.0278,
12,Atuona,PF,77.63,light rain,-9.8000,-139.0333,
14,Oyama,JP,76.59,clear sky,36.3000,139.8000,
15,Kahului,US,80.96,few clouds,20.8947,-156.4700,
18,Bengkulu,ID,76.08,light rain,-3.8004,102.2655,
22,Sechura,PE,76.12,overcast clouds,-5.5569,-80.8222,
23,Albany,US,75.36,light rain,42.6001,-73.9662,
24,Corn Island,NI,80.40,overcast clouds,12.1694,-83.0418,


## Find the nearest hotel

In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found...skipping.")
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


## Drop any rows in hotel_df where no hotel name is found

In [36]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how='any')
clean_hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Zumpahuacan,MX,77.11,overcast clouds,18.9000,-99.5500,Hotel Arbolado
5,Thompson,CA,84.36,smoke,55.7435,-97.8558,Burntwood Hotel
7,Arraial Do Cabo,BR,78.76,clear sky,-22.9661,-42.0278,Pousada Porto Praia
12,Atuona,PF,77.63,light rain,-9.8000,-139.0333,Villa Enata
14,Oyama,JP,76.59,clear sky,36.3000,139.8000,Toyoko Inn Oyama-eki Higashi-guchi No.1


## Create an output file as WeatherPy_vacation.csv

In [37]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

## Add city name, country code, weather description and max temp to info box

In [38]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


## Retrieve city data and lat/long

In [13]:
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

## Create Marker layer map with pop-up markers

In [15]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

## Take a screenshot of map and save to Vacation_Search folder as WeatherPy_vacation_map.png